In [ ]:
import sys
import os

# Get the path of the current notebook's directory and append the parent directory to the path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import importlib

import pdc_functions.assumption_utils as assumptions
import pdc_functions.data_utils as data
import pdc_functions.explore_utils as explore
import pdc_functions.pdc_1and2_utils as pdc
import pdc_functions.stats_utils as stats
from dotenv import load_dotenv
import pdc_functions.class_utils as class_utils
import pandas as pd

importlib.reload(data)

from phmlondon.snow_utils import SnowflakeConnection


In [ ]:
load_dotenv()

snowsesh = SnowflakeConnection()
snowsesh.use_database("INTELLIGENCE_DEV")
snowsesh.use_schema("AI_CENTRE_FEATURE_STORE")

In [ ]:
ldl = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.ldl_all")


Clean the ldl column first

In [ ]:
print(ldl)

In [ ]:
statin = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.statin_all")

In [ ]:
drug_table = class_utils.MedicationTable(statin)


In [ ]:
drug_table.validate_columns()  # Validate columns
drug_table.normalise_missing_values()  # Standardise representation of missing data
drug_table.convert_dates()  # Convert date columns
drug_table.validate_data_types()  # Validate data types
drug_table.clean_dose()  # Clean and interpret the 'dose' column
drug_table.calculate_covered_days()

print("All validations passed successfully!")

In [ ]:
person_drug_pdc = pdc.compute_pdc_overall(drug_table.df)

In [ ]:
person_drug_pdc = data.add_demographic_data(snowsesh, person_drug_pdc)

In [ ]:
person_drug_pdc = data.general_agg(person_drug_pdc)

In [ ]:
print(person_drug_pdc)

Join to results table

In [ ]:
data_joined = data.attach_closest_results(person_drug_pdc,ldl)

In [ ]:
print(data_joined)


In [ ]:
data_joined_2 =  data.attach_closest_results_2(person_drug_pdc,ldl)

Clean 
- remove people without at least a year of medication
- remove people with missing bloods - need at least 2 before and 2 after for average


In [ ]:
clean_data = data.cohort_exclusions(data_joined)

In [ ]:
print(clean_data)

Average the results before/after